<a href="https://colab.research.google.com/github/lwgmiller/HonoursProject-reharmonisationGAN/blob/main/ReharmonisationGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
!apt-get install python3.5

print(python_version())

In [1]:

!pip3 install torch matplotlib tqdm livelossplot gdown "pypianoroll>=1.0.2"

#!pip3 install py_midi
#import midi

     |████████████████████████████████| 5.6 MB 7.5 MB/s 
     |████████████████████████████████| 51 kB 5.2 MB/s 
     |████████████████████████████████| 793 kB 47.9 MB/s 
     |████████████████████████████████| 381 kB 56.7 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=d4562ec39a7f7cac141705c3487757699c92d6fdae6e2dbed37dcbc0ff4a3458
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [2]:
from IPython.display import clear_output
from ipywidgets import interact, IntSlider

import os, os.path, shutil
import zipfile
import random
import json
import pickle
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import torch
import pypianoroll
import pretty_midi
from pypianoroll import Multitrack, Track
from tqdm import tqdm
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot



import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import moviepy.editor as mpy
import music21
from IPython import display
import seaborn as sns

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1679360/45929032 bytes (3.7%)4079616/45929032 bytes (8.9%)6266880/45929032 bytes (13.6%)8151040/45929032 bytes (17.7%)10461184/45929032 bytes (22.8%)13377536/45929032 bytes (29.1%)14868480/45929032 bytes (32.4%)16973824/45929032 bytes (37.0%)18685952/45929032 bytes (40.7%)20627456/45929032 bytes (44.9%)22814720/45929032 bytes (49.7%)25600000/45929032 bytes (55.7%)281

In [54]:
# Data
n_tracks = 5  # number of tracks
n_pitches = 72  # number of pitches
lowest_pitch = 24  # MIDI note number of the lowest pitch
n_samples_per_song = 8  # number of samples to extract from each song in the datset
n_measures = 4  # number of measures per sample
beat_resolution = 4  # temporal resolution of a beat (in timestep)
programs = [0, 0, 0, 0]  # program number for each track
is_drums = [False, False, False, False]  # drum indicator for each track
track_names = ['Soprano', 'Alto', 'Tenor', 'Bass']  # name of each track
tempo = 100

# Training
batch_size = 16
latent_dim = 128
n_steps = 1000

# Sampling
sample_interval = 10  # interval to run the sampler (in step)
n_samples = 4

#Directories
!mkdir -p midiDataset

In [29]:
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content': ['.config', 'midiDataset', 'midi-dataset', 'drive', 'sample_data']


In [42]:
"""Midi dataset."""

from typing import Tuple
from torch import Tensor

import torch
from torch import nn
from torch.utils.data import Dataset

import numpy as np
from music21 import midi
from music21 import converter
from music21 import note, stream, duration, tempo


class MidiDataset(Dataset):
    """MidiDataset.
    Parameters
    ----------
    path: str
        Path to dataset.
    split: str, optional (default="train")
        Split of dataset.
    n_bars: int, optional (default=2)
        Number of bars.
    n_steps_per_bar: int, optional (default=16)
        Number of steps per bar.
    """

    def __init__(
        self,
        path: str,
        split: str = "train",
        n_bars: int = 2,
        n_steps_per_bar: int = 16,
    ) -> None:
        """Initialize."""
        self.n_bars = n_bars
        self.n_steps_per_bar = n_steps_per_bar
        dataset = np.load(path, allow_pickle=True, encoding="bytes")[split]
        self.data_binary, self.data_ints, self.data = self.__preprocess__(dataset)

    def __len__(self) -> int:
        """Return the number of samples in dataset."""
        return len(self.data_binary)

    def __getitem__(self, index: int) -> Tensor:
        """Return one samples from dataset.
        Parameters
        ----------
        index: int
            Index of sample.
        Returns
        -------
        Tensor:
            Sample.
        """
        return torch.from_numpy(self.data_binary[index]).float()

    def __preprocess__(self, data: np.ndarray) -> Tuple[np.ndarray]:
        """Preprocess data.
        Parameters
        ----------
        data: np.ndarray
            Data.
        Returns
        -------
        Tuple[np.ndarray]:
            Data binary, data ints, preprocessed data.
        """
        data_ints = []
        for x in data:
            skip = True
            skip_rows = 0
            while skip:
                if not np.any(np.isnan(x[skip_rows: skip_rows + 4])):
                    skip = False
                else:
                    skip_rows += 4
            if self.n_bars * self.n_steps_per_bar < x.shape[0]:
                data_ints.append(x[skip_rows: self.n_bars * self.n_steps_per_bar + skip_rows, :])
        data_ints = np.array(data_ints)
        self.n_songs = data_ints.shape[0]
        self.n_tracks = data_ints.shape[2]
        data_ints = data_ints.reshape([self.n_songs, self.n_bars, self.n_steps_per_bar, self.n_tracks])
        max_note = 83
        mask = np.isnan(data_ints)
        data_ints[mask] = max_note + 1
        max_note = max_note + 1
        data_ints = data_ints.astype(int)
        num_classes = max_note + 1
        data_binary = np.eye(num_classes)[data_ints]
        data_binary[data_binary == 0] = -1
        data_binary = np.delete(data_binary, max_note, -1)
        data_binary = data_binary.transpose([0, 3, 1, 2, 4])
        return data_binary, data_ints, data


def binarise_output(output: np.ndarray) -> np.ndarray:
    """Binarize output.
    Parameters
    ----------
    output: np.ndarray
        Output array.
    """
    max_pitches = np.argmax(output, axis=-1)
    return max_pitches


def postprocess(
    output: np.ndarray,
    n_tracks: int = 4,
    n_bars: int = 2,
    n_steps_per_bar: int = 16,
) -> stream.Score:
    """Postprocess output.
    Parameters
    ----------
    output: np.ndarray
        Output array.
    n_tracks: int, (default=4)
        Number of tracks.
    n_bars: int, (default=2)
        Number of bars.
    n_steps_per_bar: int, (default=16)
        Number of steps per bar.
    """
    parts = stream.Score()
    parts.append(tempo.MetronomeMark(number=66))
    max_pitches = binarise_output(output)
    midi_note_score = np.vstack([
        max_pitches[i].reshape([n_bars * n_steps_per_bar, n_tracks]) for i in range(len(output))
    ])
    for i in range(n_tracks):
        last_x = int(midi_note_score[:, i][0])
        s = stream.Part()
        dur = 0
        for idx, x in enumerate(midi_note_score[:, i]):
            x = int(x)
            if (x != last_x or idx % 4 == 0) and idx > 0:
                n = note.Note(last_x)
                n.duration = duration.Duration(dur)
                s.append(n)
                dur = 0
            last_x = x
            dur = dur + 0.25
        n = note.Note(last_x)
        n.duration = duration.Duration(dur)
        s.append(n)
        parts.append(s)
    return parts

In [58]:
def save_pianoroll_as_midi(pianoroll,
                  programs=programs,
                  track_names=track_names,
                  is_drums=is_drums,
                  tempo=tempo,           # in bpm
                  beat_resolution=beat_resolution,  # number of time steps
                  destination_path="/content/midiDataset/"
                  ):
    
    #pianoroll = pianoroll > 0

    # Reshape batched pianoroll array to a single pianoroll array
    pianoroll_ = pianoroll.reshape((-1, pianoroll.shape[2], pianoroll.shape[3]))

    measure_resolution = 4 * beat_resolution
    tempo_array = np.full((4 * 4 * measure_resolution, 1), tempo)

    # Create the tracks
    tracks = []
    for idx in range(pianoroll_.shape[2]):
        tracks.append(pypianoroll.BinaryTrack(
            track_names[idx], programs[idx], is_drums[idx], pianoroll_[..., idx]))

    multitrack = pypianoroll.Multitrack(
        tracks=tracks, tempo=tempo_array, resolution=beat_resolution)
        
    multitrack.write('./midiDataset/test.mid')
    print('Midi saved to ', destination_path)
    return destination_path

In [57]:
from numpy.core.fromnumeric import shape
d = np.load("/content/drive/MyDrive/Colab Notebooks/Honours code/Jsb16thSeparated.npz", allow_pickle=True, encoding = 'latin1')

path = ("/content/drive/MyDrive/Colab Notebooks/Honours code/Jsb16thSeparated.npz")

dataset = MidiDataset(path=path)

#print(dataset[0].shape)
print(d.files)
train = d['train']
test = d['test']
valid = d['valid']

#print(valid[1])
#print(dataset[1])
save_pianoroll_as_midi(train)
#print(track.shape)
#print(track)

#show_pianoroll(track)



['test', 'train', 'valid']


IndexError: ignored

In [59]:
training_data = np.load('/content/drive/MyDrive/Colab Notebooks/Honours code/train-1.npy')
print(training_data[1])

save_pianoroll_as_midi(training_data)

[[[-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  ...
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]]

 [[-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  ...
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]]

 [[-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  ...
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]]

 ...

 [[-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  ...
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]]

 [[-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  ...
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]]

 [[-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  ...
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1. -1. -1.]]]
Midi saved to  /content/midiDataset/


'/content/midiDataset/'

In [48]:
#display pianoroll

multitrack.plot()

NameError: ignored

In [ ]:

"""

length = multitrack.get_length()
print(length)
multitrack.trim(0, length)
multitrack.binarize()

multitrack.plot()


print(d.files)

for piece in d['valid']:
  
  valid.append(piece)

#print(valid)

for piece in d['train']:
  
  train.append(piece.shape)

print(train)

for piece in d['test']:
  
  test.append(piece.shape)

#print(test)



#print(piece.shape)

test1 = d['train']
print(test1.shape)

!mkdir -p data

src_file = Path("/content/drive/MyDrive/Colab Notebooks/Honours code/Jsb16thSeparated.npz")
dest_file = Path("/content/data/")
shutil.copy(src_file, dest_file)


#test1alto = pypianoroll.Track(name='Alto', program=0, is_drum=False, pianoroll=np.ndarray(shape=(208, 128), dtype=bool))
multitrack = pypianoroll.Multitrack(name='test1', resolution=24, tempo=np.ndarray(shape=(208,4),dtype=np.float16), tracks=[Track(name='Alto', program=0, is_drum=False, pianoroll=np.ndarray(shape=(228, 128), dtype=bool))])

#print(test1alto)

#test1alto.standardize()


print(multitrack)

multitrack.plot()

#length = multitrack.get_length()
#print(length)
#multitrack.trim(0, length)
#multitrack.standardize()

#print(multitrack)
#(resolution=24, tempo=array(shape=(288, 1), dtype=float64), tracks=[BinaryTrack(name='', program=0, is_drum=False, pianoroll=array(shape=(288, 128), dtype=bool))])

"""

In [ ]:
dataset_zip = Path("/content/drive/MyDrive/Colab Notebooks/Honours code/JSB Chorales.zip")
#for item in os.listdir("/content/drive/MyDrive/Colab Notebooks/Honours code/"): # loop through items in dir
    #if item.endswith(".zip"): # check for ".zip" extension
file_name = dataset_zip #os.path.abspath(item) # get full path of files
zip_ref = zipfile.ZipFile(file_name) # create zipfile object
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file
#os.remove(file_name) # delete zipped file

print(os.listdir())

['.config', 'drive', 'JSB Chorales', 'sample_data']
